In [1]:
import os
import glob

import contextily as ctx
import geopandas as gpd
import imageio
import matplotlib.pyplot as plt
import matplotlib.colors
import pandas as pd
import pygifsicle

In [2]:
flows = pd.read_csv(os.path.join('data','link_use_central_oxford.csv')) \
    .rename(columns={'edgeID': 'link', 'vehicles': 'flow'})
flows.head()

,link,hour,flow
0,47528,MIDNIGHT,16
1,47528,ONEAM,6
2,47528,TWOAM,5
3,47528,THREEAM,5
4,47528,FOURAM,18


In [3]:
flow_max = flows.flow.max()
flow_max

4556

In [4]:
unique_link_ids = flows.link.unique()
unique_link_ids

array([47528, 59425, 71095, 79227, 78868, 79103, 64522, 67418, 71430,
       64130, 70557, 69414, 76765, 79104, 58968, 70558, 60574, 59426,
       80522, 69291, 65865, 65864, 79226, 80523, 60136, 48573, 52808,
       67417, 52807, 59068, 52723, 69415, 67729, 78867, 63440, 58967,
       71429, 53203, 67730, 71094, 73222, 73221, 50858, 52722, 69114,
       67204, 59067, 57036, 65040, 77731, 72070, 65041, 69713, 53957,
       55563, 62610, 62609, 76766, 64523, 50859, 69292, 73198, 48574,
       55564, 57035, 47529, 73199, 72069, 69714, 63441, 53958, 60135,
       53204, 64129, 67203, 60575, 69113, 77730], dtype=int64)

In [5]:
len(unique_link_ids)

78

In [6]:
links = gpd.read_file(os.path.join('data','national','fullNetworkWithEdgeIDs.shp')) \
    .drop(columns=['Anode', 'Bnode', 'CP', 'iDir', 'SRefE', 'SRefN', 'Distance', 'FFspeed', 'FFtime', 'IsFerry']) \
    .rename(columns={'EdgeID': 'link'})
links.head()

,link,geometry
0,63083,"LINESTRING (334560.000 390930.000, 334790.000 ..."
1,77093,"LINESTRING (447920.000 126720.000, 448910.000 ..."
2,65231,"LINESTRING (267950.000 259740.000, 262370.000 ..."
3,54080,"LINESTRING (393720.000 344400.000, 393723.000 ..."
4,55499,"LINESTRING (250600.000 667570.000, 249480.000 ..."


In [7]:
hourly_maxes = flows.drop(columns=['link']).groupby('hour').max()
hourly_maxes.head()

,flow
hour,
EIGHTAM,4556
EIGHTPM,1237
ELEVENAM,2565
ELEVENPM,309
FIVEAM,378


In [8]:
def plot_map(hour, links, flow_max, output_filename):
    ax = links.plot(
        figsize=(10, 12),
        column='flow',
        cmap='YlOrRd',
        norm=matplotlib.colors.Normalize(vmin=0.0001, vmax=flow_max),
        legend=True,
        legend_kwds={'label': 'Flow volume', 'orientation': 'horizontal'}
    )
    plt.title('{:02d}:00'.format(hour), fontsize=16)
    ctx.add_basemap(ax, crs=links.crs)
    plt.savefig(output_filename)
    plt.close()

In [9]:
hours = [
    'MIDNIGHT',
    'ONEAM',
    'TWOAM',
    'THREEAM',
    'FOURAM',
    'FIVEAM',
    'SIXAM',
    'SEVENAM',
    'EIGHTAM',
    'NINEAM',
    'TENAM',
    'ELEVENAM',
    'NOON',
    'ONEPM',
    'TWOPM',
    'THREEPM',
    'FOURPM',
    'FIVEPM',
    'SIXPM',
    'SEVENPM',
    'EIGHTPM',
    'NINEPM',
    'TENPM',
    'ELEVENPM',
]
for hour, hour_key in enumerate(hours):
    hour_flows = flows[flows.hour == hour_key].copy()
    hour_flows = links.merge(hour_flows, on='link')
    plot_map(hour, hour_flows, flow_max, os.path.join('vis', '{:02d}.png'.format(hour)))

C:\Users\edwar\Anaconda3\envs\anp4d\lib\site-packages\PIL\Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [10]:
gif_path = os.path.join('vis', 'movie.gif')
filenames = glob.glob(os.path.join('vis','*.png'))

images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave(os.path.join(gif_path), images)

In [11]:
pygifsicle.optimize('vis\movie.gif')
